In [1]:
# -- coding: utf-8 --


import warnings
warnings.filterwarnings("ignore")
import sys
#sys.path.append('../')
sys.path.append('../../../run_came/')
from analysis_utils import ttest_plot_utils
from analysis_utils import homo_random_config as config
import os


if __name__ == '__main__':

    cfg = config._C
    #os.environ['CUDA_VISIBLE_DEVICES'] = '-1' #cfg.CAME.visible_device
    #cfg.CAME.n_top_genes = 1000
    cfg.CAME.visible_device = '-1'
    n_top_genes_list = [4000]

    cfg.CAME.species_name_list = ['Macaque', 'Mouse']
    cfg.CAME.learning_label = ['cluster_region', 'cluster_region']# #['cluster_name', 'cluster_name'] # ['region_name', 'region_name']
    cfg.CAME.annotation_name = ['region_name', 'region_name']
    
    cfg.CAME.n_top_genes = n_top_genes_list[0]
    cfg.CAME.sparse = False
    cfg.CAME.do_normalize = [True, True]
    cfg.CAME.ROOT = './Results/initial_embedding/'
    cfg.CAME.path_rawdata2 = './Data/Mouse.h5ad'
    cfg.CAME.path_rawdata1 = './Data/Macaque.h5ad'

    cfg.CAME.path_labels_2 = './Data/mouse_region_list.csv'
    cfg.CAME.path_labels_1 = './Data/macaque_region_list.csv'

    cfg.CAME.human_mouse_homo_region = './Data/mouse_macaque_homo_region.csv'
    #    ttest_plot_utils.run_came_homo_random(cfg)

    cfg.CAME.path_varmap = './Data/Macaque_Mouse_multi2multi.csv'
    cfg.CAME.path_varmap_1v1 = './Data/Macaque_Mouse_one2one.csv'

    cfg.PROCESS.path_rawdata1 = cfg.CAME.path_rawdata1
    cfg.PROCESS.path_rawdata2 = cfg.CAME.path_rawdata2

    cfg.TRAINING.n_epochs = 300

    cfg.CAME.preclustering_resolution = 6

    ttest_plot_utils.run_came_homo_random(cfg)



KeyboardInterrupt: 

In [ ]:
# -- coding: utf-8 --

import sys
sys.path.append('../../../')
from BrainAlign.code.utils import set_params
from BrainAlign.brain_analysis.configs import sr_rsc_config_binary
from BrainAlign.brain_analysis.data_utils import load_dimensions_binary
from BrainAlign.brain_analysis import process
from BrainAlign.SR_RSC import main_sr_rsc

from BrainAlign.brain_analysis.analysis_general import alignment_STs_analysis,\
    gene_comparison

from BrainAlign.brain_analysis.analysis import load_srrsc_embeddings
# from BrainAlign.brain_analysis.analysis_anatomical import anatomical_STs_analysis

# from BrainAlign.brain_analysis.analysis_genomic import genomic_analysis

from BrainAlign.brain_analysis.process import get_spatial_relation

# from BrainAlign.brain_analysis.analysis_spatial import spatial_analysis

import time

import warnings
warnings.filterwarnings("ignore")
import anndata as ad

if __name__ == '__main__':
    cfg = sr_rsc_config_binary._C

    if_train = True

    cfg.SRRSC_args.if_pretrained = False
    #cfg.freeze()

    cfg.CAME.path_rawdata1 = './Data/Macaque.h5ad'
    cfg.CAME.path_rawdata2 = './Data/Mouse.h5ad'

    # adata_1 = ad.read_h5ad(cfg.CAME.path_rawdata1)
    # print(adata_1)
    # adata_1.obs['region_name'] = adata_1.obs['gene_area']
    # adata_1 = adata_1[adata_1.obs['annotation'].isin(['Macaque_CA1', 'Macaque_CA2', 'Macaque_CA3', 'Macaque_DG'])]
    # adata_1.write_h5ad(cfg.CAME.path_rawdata1)
    # print(adata_1.obs['gene_area'].unique())
    # adata_2 = ad.read_h5ad(cfg.CAME.path_rawdata2)
    # adata_2.obs['region_name'] = adata_2.obs['gene_area']
    # adata_2 = adata_2[adata_2.obs['annotation'].isin(['Mouse_CA1', 'Mouse_CA2', 'Mouse_CA3', 'Mouse_DG'])]
    # adata_2.write_h5ad(cfg.CAME.path_rawdata2)
    # print(adata_2)
    # print(adata_2.obs['gene_area'].unique())


    cfg.CAME.ROOT = './Results/initial_embedding/Macaque-Mouse/'#Baron_mouse-Baron_human-05-23_21.33.44/ #Baron_mouse-Baron_human-01-03_19.40.04
    cfg.CAME.figdir = cfg.CAME.ROOT + 'figs/'

    cfg.CAME.homo_region_file_path = './Data/mouse_macaque_homo_region.csv'
    cfg.CAME.labels_dir = './Data/'
    
    cfg.CAME.path_labels_2 = './Data/mouse_region_list.csv'
    cfg.CAME.path_labels_1 = './Data/macaque_region_list.csv'
    #    ttest_plot_utils.run_came_homo_random(cfg)

    cfg.CAME.path_varmap = './Data/Macaque_Mouse_multi2multi.csv'
    cfg.CAME.path_varmap_1v1 = './Data/Macaque_Mouse_one2one.csv'

    cfg.CAME.human_mouse_homo_region = './Data/mouse_macaque_homo_region.csv'


    cfg.BrainAlign.dsnames = ['Macaque', 'Mouse']
    cfg.BrainAlign.homo_region_num = 4

    cfg.BrainAlign.embedding_type = 'came' #came
    cfg.BrainAlign.embedding_pca_dim = 30


    cfg.BrainAlign.dataset = 'macaque_mouse_binary'
    cfg.BrainAlign.result_save_folder = './Results'
    cfg.BrainAlign.experiment_time = '' #'2024-03-12_18-38-54'#time.strftime("%Y-%m-%d_%H-%M-%S")
    cfg.BrainAlign.result_save_path = cfg.BrainAlign.result_save_folder + cfg.BrainAlign.experiment_time
    cfg.BrainAlign.embeddings_file_path = cfg.BrainAlign.result_save_path + "/embeds/"
    cfg.BrainAlign.DATA_PATH = cfg.BrainAlign.result_save_path + '/data/'

    cfg.SRRSC_args.save_path = cfg.BrainAlign.result_save_path
    cfg.SRRSC_args.data_path = cfg.BrainAlign.DATA_PATH

    S, M, binary_S, binary_M, binary_H, binary_V = load_dimensions_binary(cfg.CAME.ROOT + 'datapair_init.pickle')

    cfg.BrainAlign.S = S  # 21749 + 3682
    cfg.BrainAlign.M = M  # 4035 + 6507

    cfg.BrainAlign.binary_S = binary_S  # ,
    cfg.BrainAlign.binary_M = binary_M  # 1034  # 4035
    cfg.BrainAlign.binary_H = binary_H  # 1004  # 6507
    cfg.BrainAlign.binary_V = binary_V

    cfg.BrainAlign.node_relation = 'spatial_2d'  # 'spatial' # knn
    cfg.BrainAlign.spatial_node_neighbor = 4 # set it to 4 for slice data, 5 for 3d data

    cfg.ANALYSIS.umap_neighbor = 30  # 30
    cfg.ANALYSIS.mouse_umap_neighbor = 30  # 40
    cfg.ANALYSIS.human_umap_neighbor = 30

    cfg.ANALYSIS.umap_marker_size = 5  # 40
    cfg.ANALYSIS.mouse_umap_marker_size = 5  # 40
    cfg.ANALYSIS.human_umap_marker_size = 10  # 40

    cfg.ANALYSIS.genes_umap_neighbor = 15#15

    cfg.ANALYSIS.sample_cluster_num = 60

    cfg.ANALYSIS.cut_ov = 0

    cfg.SRRSC_args.lr = 0.01
    cfg.SRRSC_args.patience = 40

    cfg.SRRSC_args.out_ft = 128

    cfg.SRRSC_args.drop_prob = 0.6
    cfg.SRRSC_args.lamb = 0.5  # coefficient for the losses in node task
    cfg.SRRSC_args.lamb_lp = 1.0  # coefficient for the losses in link task
    cfg.SRRSC_args.margin = 0.5 #0.8

    cfg.BrainAlign.fig_format = 'png'

    cfg.SRRSC_args.isAtt = True#True#True
    cfg.SRRSC_args.isBias = True

    cfg.SRRSC_args.lamb = 0.5  # coefficient for the losses in node task
    cfg.SRRSC_args.lamb_lp = 1.0  # coefficient for the losses in link task
    cfg.SRRSC_args.margin = 0.8  # coefficient for the margin loss

    cfg.SRRSC_args.train_ratio = 0.5
    cfg.SRRSC_args.validation_ratio = 0.45
    cfg.SRRSC_args.test_ratio = 0.05

    cfg.SRRSC_args.lr_stepsize = 30
    cfg.SRRSC_args.lr_gamma = 0.5

    cfg.SRRSC_args.nb_epochs = 200

    print('--------------------------Config:', cfg)
    if if_train:
        logger = process.get_srrsc_input(cfg)
        print('Training BrainAlign...')
        main_sr_rsc.run_srrsc(cfg, logger)

    print('Analysis of BrainAlign embeddings...')

    load_srrsc_embeddings(cfg)  # (21749, 128)
    #alignment_STs(cfg)

    alignment_STs_analysis_obj = alignment_STs_analysis(cfg)
    #alignment_STs_analysis_obj.forward()
    alignment_STs_analysis_obj.init_data()
    alignment_STs_analysis_obj.experiment_1_cross_species_clustering()

    alignment_STs_analysis_obj.experiment_2_umap_evaluation()
    
    ##alignment_STs_analysis_obj.experiment_2_umap_diagram()
    alignment_STs_analysis_obj.experiment_3_homo_random()
    alignment_STs_analysis_obj.experiment_3_1_homo_random_beforeAlign
    ##self.experiment_3_homo_random()
    
    alignment_STs_analysis_obj.experiment_4_cross_species_genes_analysis()

    alignment_STs_analysis_obj.experiment_5_genes_homo_random()
    # alignment_STs_analysis_obj.experiment_6_brain_region_classfier()
    #alignment_STs_analysis_obj.experiment_7_align_cross_species()
    #alignment_STs_analysis_obj.experiment_7_1_align_cross_species_split()
    # alignment_STs_analysis_obj.experiment_8_cross_evaluation_aligment_cluster()
    # alignment_STs_analysis_obj.experiment_8_1_cross_aligment_cluster_merge_lines()
    # alignment_STs_analysis_obj.experiment_9_name_clusters()
    #alignment_STs_analysis_obj.experiment_9_1_region_cluster_heatmap()
    #alignment_STs_analysis_obj.experiment_10_check_alignment()
    #alignment_STs_analysis_obj.experiment_10_1_check_alignment_plot()
    #alignment_STs_analysis_obj.experiment_11_paga()

    # alignment_STs_analysis_obj.experiment_14_clusters_alignment()
    #
    # alignment_STs_analysis_obj.experiment_15_regions_alignment()
    # alignment_STs_analysis_obj.experiment_15_1_isocortex_clusters(if_load=True)
    #
    # alignment_STs_analysis_obj.experiment_16_thalamus_clusters(if_load=True)
    # alignment_STs_analysis_obj.experiment_17_hypothalamus_clusters(if_load=True)
    # alignment_STs_analysis_obj.experiment_18_pons_clusters(if_load=True)

    #alignment_STs_analysis_obj.experiment_18_pons_marker_gene()

    #alignment_STs_analysis_obj.experiment_19_Midbrain_hypothalamus(if_load=True, topk=30)

    #alignment_STs_analysis_obj.experiment_7_align_cross_species()
    #
    #alignment_STs_analysis_obj.experiment_4_cross_species_genes_analysis()
    #alignment_STs_analysis_obj.experiment_4_1_genes_statistics()
    #alignment_STs_analysis_obj.experiment_8_cross_evaluation_aligment_cluster()
    #alignment_STs_analysis_obj.experiment_9_name_clusters()
    #alignment_STs_analysis_obj.experiment_10_check_alignment()

    #alignment_STs_analysis_obj.forward()
    #genomic_analysis_obj = genomic_analysis(cfg)
    #genomic_analysis_obj.experiment_1_th_clusters_deg()
    #genomic_analysis_obj.experiment_2_clusters_deg()
    #genomic_analysis_obj.experiment_2_1_homoregions_deg()
    #genomic_analysis_obj.experiment_3_regions_gene_module()
    #genomic_analysis_obj.experiment_3_1_regions_gene_module_analysis()
    #genomic_analysis_obj.experiment_4_clusters_gene_module()
    #genomic_analysis_obj.experiment_5_deg_homo_distribution()
    #genomic_analysis_obj.experiment_6_regions_gene_module_average_exp()
    #genomic_analysis_obj.experiment_6_1_regions_gene_module_cross()
    #genomic_analysis_obj.experiment_7_clusters_gene_module_average_exp()
    #genomic_analysis_obj.experiment_7_1_clusters_gene_module_cross()
    #genomic_analysis_obj.experiment_8_homoregions_go_terms_overlap()
    #genomic_analysis_obj.experiment_8_1_plot_heatmap()
    #genomic_analysis_obj.experiment_8_2_homoregions_go_terms_cross()

    #genomic_analysis_obj.experiment_9_clusters_go_terms_overlap()

    #genomic_analysis_obj.experiment_9_1_plot_heatmap()

    #genomic_analysis_obj.experiment_10_binary_clustering()

    #gene_comparison(cfg)
    #anatomical_STs_analysis_obj = anatomical_STs_analysis(cfg)
    #anatomical_STs_analysis_obj.forward()
    #spatial_analysis_obj = spatial_analysis(cfg)
    #spatial_analysis_obj.experiment_1_spatial_hippocampal()
    #spatial_analysis_obj.forward()
    #spatial_analysis_obj.experiment_2_spatial_isocortex()
    #spatial_analysis_obj.experiment_3_spatial_clusters()



